## Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

### 1. Изучение общей информации о данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.dummy import DummyRgressor

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<div class="alert alert-block alert-warning">
<b>Изменения:</b> добавила метод
</div>

Файл с данными содержит более 3 тысяч строк и 5 столбцов. Все ячейки заполнены.


### 2. Разбейте данные на выборки

In [3]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_temp, features_valid, target_temp, target_valid = train_test_split(
    features, target, test_size=0.2, random_state=12345) 
features_train, features_test, target_train, target_test = train_test_split(
    features_temp, target_temp, test_size=0.25, random_state=12345) 

print('Размеры полученных наборов:')
print('тренировочный - ', len(features_train))
print('валидационный - ', len(features_valid))
print('тестовый - ', len(features_test))

Размеры полученных наборов:
тренировочный -  1928
валидационный -  643
тестовый -  643


Сначала разбиваем данные на две части - времееную и валидационную(20%). Затем временную часть делим на тренировочную(60%) и тестовую(20%) выборки. Целевой признак - is_ultra.

### 3. Исследуйте модели

#### Для начала возьмём алгоритм классификации случайный лес 

In [4]:
best_model_forest = None
best_result = 0
# исследуем качество на разном количестве деревьев, высота дерева 7 
for est in range(1, 11):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth) # обучение модели
        model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
        result = model.score(features_valid, target_valid) # считаем качество модели на валидационной выборке
        if result > best_result:
            best_model_forest = model
            best_result = result
        

print("Количество деревьев:", best_model_forest.n_estimators)
print("Глубина каждого дерева:", best_model_forest.max_depth)
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Количество деревьев: 3
Глубина каждого дерева: 7
Accuracy наилучшей модели на валидационной выборке: 0.80248833592535


#### Исследуем модель решающего дерева

In [5]:
best_model_tree = None
best_result = 0
# исследуем качество модели с разной глубиной дерева
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train) # обучаем модель
    predictions = model.predict(features_valid) # получаем предсказания модели
    result = accuracy_score(target_valid, predictions) # считаем качество модели
    if result > best_result:
        best_model_tree = model
        best_result = result
        

print("Оптимальная глубина дерева:", best_model_tree.max_depth)
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Оптимальная глубина дерева: 5
Accuracy наилучшей модели на валидационной выборке: 0.7884914463452566


Качество модели решающего дерева для этого набора данных немного ниже чем у случайного леса.


### 4. Проверьте модель на тестовой выборке

In [6]:
predictions_test = best_model_tree.predict(features_test)
accuracy_test = accuracy_score(target_test, predictions_test)
result = best_model_forest.score(features_test, target_test) 
print('Accuracy модели на тестовой выборке:')
print('Решающее дерево - ', accuracy_test)
print('Случайный лес - ', result)

Accuracy модели на тестовой выборке:
Решающее дерево -  0.7589424572317263
Случайный лес -  0.776049766718507


Проверка на тестовой выборке показала, что модели показывают примерно одинаковое качество.

По моему мнению для имеющегося набора данных больше подходит модель решающего дерева. Она более простая и быстрая чем модель случайного леса.